In [ ]:
import numpy as np
import pandas as pd
import gc
from tqdm import *
import os
import copy
import sys
import time
from collections import Counter, deque
from multiprocessing import Pool, cpu_count

import transforms3d

from sklearn.manifold import TSNE, LocallyLinearEmbedding
from sklearn.cluster import MeanShift

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipyvolume as ipv
import open3d as o3d

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# %matplotlib inline
%matplotlib notebook

In [ ]:
ATM_PATH = "/home/sabyasachi/Projects/ati/"
DATA_PATH = os.path.join(ATM_PATH, "data", "data", "datasets")

# IISC_DATA_PATH = os.path.join(DATA_PATH, "IISC")
# EXP_PATH = os.path.join(IISC_DATA_PATH, "2019-06-12","10-00-14-P1-6-auto-ccw_5loops_0.6_no_numba")
IISC_DATA_PATH = os.path.join(DATA_PATH, "Carla")
EXP_PATH = os.path.join(IISC_DATA_PATH, "dynamic", "set1_just_for_running")

LIDAR_PCD_PATH = os.path.join(EXP_PATH, "_out")

# MAP_FILE = 'map_default_r.pcd'
# POSE_FILE = 'pose_default_r.json'

OUTPUT_DIR = "./carla_dynamic_set1_just_for_running_tenth_frame"

In [ ]:
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
FIRST_PCD = 2
FINAL_PCD = 4628
# FIRST_PCD = 50
# FINAL_PCD = 750
# FIRST_PCD = 2
# FINAL_PCD = 805

EVERY_NTH_COUNT = 10
VOXEL_SZ = 0.2
# MAKE_2D = False
# APPLY_CLUSTER = False

# plt.rcParams['figure.figsize'] = [7, 7]

In [ ]:
def filter_pcd(old_pcd,
               apply_downsample = True,
               downsample_voxel_size = VOXEL_SZ,
               
               apply_outlier_removal = True,
               downsample_radius = 1,
               downsample_neighbors = 20,
               
               apply_crop = True,
               crop_min_arr = np.array([-100,-100,0]),
               crop_max_arr = np.array([100,100,100]),
               
               apply_cluster = False,
               cluster_neighbours = 30,
               cluster_labels = 2):
    np.random.seed(0)
    pcd = copy.deepcopy(old_pcd)
    
    if apply_outlier_removal:
        denser_pcd, ind = o3d.geometry.radius_outlier_removal(pcd,
                                                              nb_points = downsample_neighbors,
                                                              radius    = downsample_radius)
        pcd = denser_pcd
    
    if apply_downsample:
        voxel_down_pcd = o3d.geometry.voxel_down_sample(pcd, voxel_size = downsample_voxel_size)
        pcd = voxel_down_pcd
    
    if apply_crop:
        cropped_pcd = o3d.geometry.crop_point_cloud(pcd, crop_min_arr, crop_max_arr)
        pcd = cropped_pcd

    if apply_cluster:
        few_pts = np.asarray(pcd.points)
        try:
            few_pts_reduced = LocallyLinearEmbedding(n_neighbors=cluster_neighbours, n_components=2).fit_transform(few_pts)
        except Exception as e:
            try:
                few_pts_reduced = LocallyLinearEmbedding(n_neighbors=cluster_neighbours, n_components=2, eigen_solver='dense').fit_transform(few_pts)
            except Exception as e:
                few_pts_reduced = few_pts
        clf = MeanShift().fit(few_pts_reduced)
        pcd.points = o3d.utility.Vector3dVector(few_pts[clf.labels_ < cluster_labels])
    
    return pcd

def make_2d(pcd):
    new_pcd = copy.deepcopy(pcd)
    new_pts = np.concatenate([np.asarray(pcd.points)[:,:-1],np.zeros((len(pcd.points),1))], axis=1)
    new_pcd.points = o3d.utility.Vector3dVector(new_pts)
    return new_pcd

def read_pcd(pcd_id):
#     prefix = "".join(["0" for _ in range(3 - len(str(pcd_id)))])
    pcd_file = str(pcd_id) + ".ply"
    pcd = o3d.io.read_point_cloud(os.path.join(LIDAR_PCD_PATH, pcd_file))
    return pcd

def draw_pcd(pcd, where='mat_2d'):    
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], s=1)
        plt.show()
        
def draw_registration_result(src_pcd, dst_pcd, x_pt, y_pt, theta):    
    src_pcd_tmp = copy.deepcopy(src_pcd)
    dst_pcd_tmp = copy.deepcopy(dst_pcd)
    
    src_pcd_tmp.paint_uniform_color([1, 0, 0])  # red source
    dst_pcd_tmp.paint_uniform_color([0, 0, 1])  # blue target
    
    transform_mat = pose2matrix([x_pt, y_pt, 0], [0,0,theta])
    dst_pcd_tmp.transform(transform_mat)
    
    visualizer = o3d.JVisualizer()
    visualizer.add_geometry(src_pcd_tmp)
    visualizer.add_geometry(dst_pcd_tmp)
    visualizer.show()

In [ ]:
file_idx = 0
for pcd_idx in tqdm_notebook(np.arange(start=FIRST_PCD, stop=FINAL_PCD+1, step=EVERY_NTH_COUNT)):
    # Read pcd
    some_pcd = read_pcd(pcd_idx)
    
    # Remove zero points
    some_arr = np.asarray(some_pcd.points)
    some_arr = np.array([(x,y,z) for x, y, z in some_arr if not (x == 0 and y == 0 and z == 0)])
    some_pcd.points = o3d.utility.Vector3dVector(some_arr)
    
    # Remove outliers and crop ground circle points
    some_pcd = filter_pcd(some_pcd,
                          apply_downsample = False,
                          apply_outlier_removal = True,
                          apply_crop = True,
                          apply_cluster = False)
    
    # Make it 2d
    some_pcd = make_2d(some_pcd)
    
    # Downsample
    for _ in range(3):
        some_pcd = filter_pcd(some_pcd,
                              apply_downsample = True,
                              apply_outlier_removal = False,
                              apply_crop = False,
                              apply_cluster = False)
    
    # Set filename
    prefix = "".join(["0" for _ in range(3 - len(str(file_idx)))])
    single_pcd_fname = prefix + str(file_idx) + ".pcd"
    single_pcd_path = os.path.join(OUTPUT_DIR, single_pcd_fname)
    
    # Save pcd
    o3d.io.write_point_cloud(single_pcd_path, some_pcd)
    
    file_idx  += 1